In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import pandas as pd
import numpy as np

import time

import torch

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
def fit(model, save_dir, train_set, test_set, forget_set=None):
    
    os.makedirs(save_dir, exist_ok=True)
    
    optimizer = torch.optim.Adam(model.parameters())
    error = torch.nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        accs = list()
        
        train_time = 0
        
        model.train()
        
        start_time = time.time()
        
        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            loss.backward()
            optimizer.step()
            
            train_time += time.time() - start_time
            
            predicted = torch.argmax(output.data, dim=-1)
            accs.append((predicted == y).float().mean().detach().cpu().numpy())
            
            start_time = time.time()
            
        train_times.append(train_time)
        train_accs.append(np.mean(accs))
        
        # eval
            
        model.eval()
        with torch.no_grad():
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            if forget_set is not None:

                x, y = forget_set.tensors
            
                accs = list()

                for i in range(0, x.shape[0], BATCH_SIZE):

                    output = model(x[i:i+BATCH_SIZE].cuda())

                    predicted = torch.argmax(output.data, dim=-1)
                    accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

                forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))
        

    return train_times, train_accs, test_accs, forget_accs

# origin train

In [5]:
train_set, test_set = load_original_dataset(DATA_DIR)

In [6]:
model = CNN().cuda()

In [7]:
# save
torch.save(model.state_dict(), 'weights/init.pt')

In [8]:
train_times, train_accs, test_accs, _ = fit(model, 'weights/original', train_set, test_set)

In [9]:
df = pd.DataFrame(zip(train_times, train_accs, test_accs), columns=['train_time', 'train_acc', 'test_acc'])
df['epoch'] = range(1, EPOCHS+1)
df.set_index('epoch', inplace=True)

df.to_csv('results/original.csv')

df

,train_time,train_acc,test_acc
epoch,,,
1,12.413962,0.868583,0.973143
2,11.085280,0.944733,0.980931
3,10.715224,0.954433,0.989317
4,11.184714,0.957367,0.986821
5,11.101995,0.959433,0.985723


# naive retrain

In [10]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
        
    model.load_state_dict(torch.load('./weights/init.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/naive_retrain/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/naive_retrain.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       11.065428   0.857018  0.972943    0.972039
           2       11.427742   0.940531  0.983327    0.983553
           3       11.147157   0.950313  0.985723    0.980263
           4       12.055764   0.952418  0.985923    0.983553
           5       10.988493   0.958025  0.985623    0.981908
10         1        9.890369   0.854957  0.973942    0.976396
           2        9.809762   0.939964  0.981230    0.978890
           3        9.869044   0.950022  0.981929    0.979721
           4        9.747755   0.953523  0.986122    0.984874
           5        9.809713   0.957302  0.987720    0.985539
20         1        9.203177   0.846146  0.975839    0.972750
           2        8.865873   0.940021  0.981030    0.977250
           3        8.883193   0.948625  0.981430    0.978000
           4        8.964614   0.953500  0.984625    0.982083
           5        9.964748   0.957479  0.987021    0.984500
30         1        7.790184   0.833794  0.972444    0.971192
           2        7.562800   0.937309  0.979333    0.977020
           3        7.678257   0.946742  0.982228    0.979518
           4        8.073815   0.952887  0.983427    0.981239
           5        7.733404   0.955626  0.985823    0.982571
40         1        6.567179   0.825694  0.972244    0.970125
           2        6.508478   0.932250  0.978235    0.976333
           3        6.338063   0.944167  0.985224    0.981125
           4        6.731266   0.948944  0.982328    0.979125
           5        6.758206   0.956194  0.986821    0.981917
50         1        5.289086   0.815635  0.970447    0.967784
           2        5.597664   0.930563  0.981030    0.978345
           3        5.557550   0.943670  0.979034    0.977479
           4        5.662526   0.946872  0.976038    0.973181
           5        5.625827   0.953642  0.982827    0.980810
60         1        4.396187   0.737250  0.967252    0.964139
           2        4.404668   0.920250  0.971845    0.967278
           3        4.580806   0.937167  0.981729    0.978639
           4        4.822374   0.945250  0.979433    0.974611
           5        4.400605   0.945417  0.980931    0.978556
70         1        3.170623   0.725923  0.942592    0.942665
           2        3.333591   0.888123  0.964157    0.962038
           3        3.223794   0.917816  0.968351    0.964633
           4        3.229980   0.932051  0.973043    0.971820
           5        3.314745   0.941003  0.973742    0.971106
80         1        2.032088   0.696583  0.940196    0.934521
           2        2.153865   0.896833  0.959864    0.957521
           3        2.216328   0.918583  0.972045    0.964625
           4        2.103795   0.934667  0.975539    0.972583
           5        2.099107   0.944917  0.973642    0.970646
90         1        0.681389   0.543449  0.886182    0.870612
           2        0.966778   0.840241  0.928714    0.924152
           3        0.982314   0.889873  0.945986    0.941480
           4        0.936354   0.917781  0.957668    0.953014
           5        0.940168   0.929646  0.966054    0.961067
99         1        0.061015   0.125000  0.217652    0.218060
           2        0.074171   0.208333  0.276158    0.274687
           3        0.069081   0.328125  0.513478    0.507455
           4        0.068599   0.491319  0.710763    0.707223
           5        0.068632   0.633681  0.711162    0.710134